In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [2]:
X = np.genfromtxt("data/X_train.txt")
Y = np.genfromtxt("data/Y_train.txt")
X, Y = ml.shuffleData(X, Y)
Xte = np.genfromtxt("data/X_test.txt")

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.75)

In [48]:
neigh = KNeighborsClassifier(n_neighbors = 7,algorithm='auto',leaf_size = 30, metric = 'minkowski', metric_params=None, n_jobs=1,p=1, weights='uniform')
neigh.fit(Xtr, Ytr)

#prediction = neigh.predict_proba(Xtr)
#predictions = np.mean(np.array([prediction]), axis=0)

#false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, predictions[:, 1])
#roc_auc = auc(false_positive_rate, true_positive_rate)
#print(roc_auc)




KNeighborsClassifier(n_jobs=1, n_neighbors=7, p=1)

In [3]:
#Randforest
rf = RandomForestClassifier(n_estimators=500, max_depth=18, max_samples=0.9)
rf.fit(X, Y)

#prediction = rf.predict_proba(X)
#predictions = np.mean(np.array([prediction]), axis=0)

#false_positive_rate, true_positive_rate, thresholds = roc_curve(Y, predictions[:, 1])
#roc_auc = auc(false_positive_rate, true_positive_rate)
#print(roc_auc)

RandomForestClassifier(max_depth=18, max_samples=0.9, n_estimators=500)

In [4]:
#GradientBoost

gb = GradientBoostingClassifier(n_estimators=800, learning_rate=0.1, max_depth=9, random_state=0, verbose=1).fit(X, Y)


      Iter       Train Loss   Remaining Time 
         1           1.2607           14.76m
         2           1.2395           14.78m
         3           1.2222           14.76m
         4           1.2065           14.63m
         5           1.1927           14.58m
         6           1.1814           14.54m
         7           1.1713           14.48m
         8           1.1620           14.49m
         9           1.1540           14.47m
        10           1.1468           14.46m
        20           1.0991           14.29m
        30           1.0689           13.98m
        40           1.0489           13.77m
        50           1.0316           13.58m
        60           1.0177           13.48m
        70           1.0067           13.44m
        80           0.9969           13.38m
        90           0.9877           13.34m
       100           0.9777           13.23m
       200           0.8974           11.48m
       300           0.8359            9.63m
       40

In [5]:
from sklearn.ensemble import VotingClassifier

estimators=[("rforest", rf), ("gboost", gb)]
ensemble = VotingClassifier(estimators, voting="soft", verbose = True)
ensemble.fit(X,Y)

[Voting] .................. (1 of 2) Processing rforest, total= 2.3min
      Iter       Train Loss   Remaining Time 
         1           1.2607           14.50m
         2           1.2395           14.56m
         3           1.2222           14.58m
         4           1.2065           14.47m
         5           1.1927           14.41m
         6           1.1814           14.39m
         7           1.1713           14.35m
         8           1.1620           14.34m
         9           1.1540           14.30m
        10           1.1468           14.32m
        20           1.0991           14.17m
        30           1.0689           13.89m
        40           1.0489           13.72m
        50           1.0316           13.54m
        60           1.0177           13.45m
        70           1.0067           13.41m
        80           0.9969           13.34m
        90           0.9877           13.29m
       100           0.9777           13.20m
       200           0.8974 

VotingClassifier(estimators=[('rforest',
                              RandomForestClassifier(max_depth=18,
                                                     max_samples=0.9,
                                                     n_estimators=500)),
                             ('gboost',
                              GradientBoostingClassifier(max_depth=9,
                                                         n_estimators=800,
                                                         random_state=0,
                                                         verbose=1))],
                 verbose=True, voting='soft')

In [6]:
prediction = ensemble.predict_proba(Xtr)
predictions = np.mean(np.array([prediction]), axis=0)

false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, predictions[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
    
    
    
prediction = ensemble.predict_proba(Xva)
predictions = np.mean(np.array([prediction]), axis=0)

false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, predictions[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
    
    
np.savetxt('Ensemble2_predictions.txt',
           np.vstack((np.arange(len(predictions)), predictions[:, 1])).T,
           '%d, %.2f', header='ID,Prob1', comments='', delimiter=',')

0.9473789602858673
0.9478198239864548
